In [1]:
import tensorflow

print(tensorflow.__version__)

2.6.0


# Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)

In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import re

데이터 가져오기

In [4]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
48168,Do you play in a band?,Joues-tu dans une fanfare ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
69305,Your house is fantastic.,Votre maison est fantastique.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
133555,Why don't you take your coat off?,Pourquoi n'ôtez-vous pas votre manteau ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
164128,We could see enemy ships on the horizon.,Nous pouvions voir des navires ennemis à l'hor...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
139205,You expect too much of your child.,Tu en demandes trop à ton enfant.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [5]:
lines 

,eng,fra,cc
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
197458,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
197459,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
197460,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
197461,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [7]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
3834,Take a bite.,Prends une bouchée !
30710,I'm still in shock.,Je suis encore sous le choc.
13188,Be more precise.,Soit plus précis.
10230,Here is my key.,Voici ma clef.
21766,What a nice bike!,Quel beau vélo !


In [8]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
4155,We love you.,\t Nous vous aimons. \n
14907,I've got a plan.,\t Je dispose d'un plan. \n
3964,Tom is back.,\t Tom est de retour. \n
8518,Take a number.,\t Prends un numéro ! \n
3854,Talk slowly.,\t Parlez lentement. \n


In [9]:
eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 4, 7], [19, 4, 7], [19, 4, 7]]

In [10]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[9, 1, 19, 5, 1, 31, 1, 10],
 [9, 1, 15, 5, 12, 16, 28, 2, 13, 1, 10],
 [9, 1, 2, 7, 1, 12, 8, 11, 4, 2, 1, 31, 1, 10]]

In [20]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73


In [21]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 19
프랑스어 시퀀스의 최대 길이 61


In [13]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 19
프랑스어 시퀀스의 최대 길이 61


In [14]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [15]:
print(decoder_input[:3])
print(decoder_target[:3])

[[9, 1, 19, 5, 1, 31, 1], [9, 1, 15, 5, 12, 16, 28, 2, 13, 1], [9, 1, 2, 7, 1, 12, 8, 11, 4, 2, 1, 31, 1]]
[[1, 19, 5, 1, 31, 1, 10], [1, 15, 5, 12, 16, 28, 2, 13, 1, 10], [1, 2, 7, 1, 12, 8, 11, 4, 2, 1, 31, 1, 10]]


In [16]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 19)
프랑스어 입력데이터의 크기(shape) : (33000, 61)
프랑스어 출력데이터의 크기(shape) : (33000, 61)


In [17]:
print(encoder_input[0])

[19  4  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [18]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 19, 51)
프랑스어 입력데이터의 크기(shape) : (33000, 61, 73)
프랑스어 출력데이터의 크기(shape) : (33000, 61, 73)


In [19]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 19, 51)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 61, 73)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 61, 73)


In [22]:
encoder_input_train = encoder_input[:30000]
decoder_input_train = decoder_input[:30000]
decoder_target_train = decoder_target[:30000]

encoder_input_test = encoder_input[30000:]
decoder_input_test = decoder_input[30000:]
decoder_target_test = decoder_target[30000:]

In [24]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 19, 51)
(30000, 61, 73)
(30000, 61, 73)
(3000, 19, 51)
(3000, 61, 73)
(3000, 61, 73)


모델 훈련

필요한 라이브러리 import

In [25]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [26]:
# LSTM의 출력 차원
latent_dim = 32

In [27]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

디코더를 설계

In [28]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

디코더의 출력층 설계

In [29]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [30]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 51)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 315392      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [31]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
235/235 [==============================] - 30s 19ms/step - loss: 1.1394 - val_loss: 0.9600
Epoch 2/50
235/235 [==============================] - 4s 15ms/step - loss: 0.7054 - val_loss: 0.7626
Epoch 3/50
235/235 [==============================] - 4s 15ms/step - loss: 0.5926 - val_loss: 0.6826
Epoch 4/50
235/235 [==============================] - 4s 15ms/step - loss: 0.5257 - val_loss: 0.6337
Epoch 5/50
235/235 [==============================] - 4s 15ms/step - loss: 0.4763 - val_loss: 0.5891
Epoch 6/50
235/235 [==============================] - 4s 15ms/step - loss: 0.4401 - val_loss: 0.5501
Epoch 7/50
235/235 [==============================] - 4s 15ms/step - loss: 0.4113 - val_loss: 0.5286
Epoch 8/50
235/235 [==============================] - 4s 15ms/step - loss: 0.3881 - val_loss: 0.5053
Epoch 9/50
235/235 [==============================] - 4s 15ms/step - loss: 0.3686 - val_loss: 0.4856
Epoch 10/50
235/235 [==============================] - 4s 15ms/step - loss: 0.3523 - val_l

모델 테스트하기

인코더 정의

In [32]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 51)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 315392    
Total params: 315,392
Trainable params: 315,392
Non-trainable params: 0
_________________________________________________________________


디코더 설계

In [33]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

디코더 출력층 재설계

In [34]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 input_3[0][0]              

In [38]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [41]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [42]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장:  Bouge ! 
번역기가 번역한 문장:  va-t’en ! 
-----------------------------------
입력 문장: Hello!
정답 문장:  Bonjour ! 
번역기가 번역한 문장:  bonjour ! 
-----------------------------------
입력 문장: Got it?
정답 문장:  T'as capté ? 
번역기가 번역한 문장:  compris ? 
-----------------------------------
입력 문장: Hang on.
정답 문장:  Tiens bon ! 
번역기가 번역한 문장:  tiens bon ! 
-----------------------------------
입력 문장: Here's $5.
정답 문장:  Voilà cinq dollars. 
번역기가 번역한 문장:  tue vous ! 


# 단어 Level로 번역기 업그레이드하기

In [43]:
import tensorflow

print(tensorflow.__version__)

2.6.0


시작

In [44]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import re

In [45]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
42967,I saw you doing that.,Je vous ai vu faire cela.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
99991,You can't give up like this.,Tu ne peux pas abandonner comme ça.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
137052,Keep it in mind for the next time.,Garde-le en tête pour la prochaine fois !,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
53497,We will not surrender.,Nous ne nous rendrons pas.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
183066,Tom told me he didn't take French in high school.,Tom m'a dit qu'il n'avait pas pris français au...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [46]:
lines = lines[['eng', 'fra']][:33000] # 3.3만개의 샘플
lines.sample(5)

,eng,fra
20645,The well ran dry.,Le puits s'est asséché.
8275,May I go home?,Puis-je aller chez moi ?
6064,Tom followed.,Tom a suivi.
6807,Do you get it?,Pigez-vous ?
7486,I need a coat.,Il me faut un manteau.



정제, 정규화, 전처리(영어, 프랑스어 둘다)

In [47]:
lines.sample(6)

,eng,fra
28058,You're productive.,Vous êtes productifs.
9170,We were bored.,Nous nous ennuyions.
25347,Let's take a vote.,Votons.
25707,She looked lonely.,Elle avait l'air seule.
11856,The birds sang.,Les oiseaux chantaient.
20969,Tom clearly lied.,Tom a manifestement menti.


전처리 - 구두점 나누기, 소문자로 만들기

In [48]:
def preprocess_sentence(sentence) :
    
    # 전처리 부분

    # 구두점을 단어와 분리를 시켜본다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-zA-Z!.?]+", r" ", sentence)
    sentence = re.sub(r"\s+", r" ", sentence)
#     sentence = re.sub(r'[" "]+', " ", sentence)
    
    # 모두 소문자로 변환
    sentence = sentence.lower()
    
    return sentence

lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocess_sentence(x))
lines.sample(5)

,eng,fra
21858,what will you do ?,que feras tu ?
8998,tom was upset .,tom tait contrari .
25149,it s all nonsense .,rien n a de sens .
14220,i know it hurts .,je sais que a fait mal .
8522,take a shower .,va prendre une douche !


시작토큰과 종료토큰 추가하기


프랑스어에만 추가한다. (디코더 언어) decoder_input에는 시작태그만, decoder_target에는 종료태그만 남아야 한다.

In [49]:
### 시작 토큰과 종료 토큰 추가
sos_token = '<sos>'
eos_token = '<eos>'
lines.fra = lines.fra.apply(lambda x : sos_token + ' ' + x + ' ' + eos_token)
print('전체 샘플의 수 : ', len(lines))
lines.sample(5)

전체 샘플의 수 :  33000


,eng,fra
27620,who s your father ?,<sos> lequel est votre p re ? <eos>
16182,tom is a muslim .,<sos> tom est musulman . <eos>
13261,can you hear us ?,<sos> parvenez vous nous entendre ? <eos>
27986,you re a survivor .,<sos> tu es un survivant . <eos>
7831,i m listening .,<sos> j coute . <eos>


단어 단위 토큰화

Tokenizer의 인자 중 char_level은 default값으로 False이다, 이 인자를 True로 사용한다면 글자 단위의 토큰화를 수행

In [50]:
eng_tokenizer = Tokenizer(filters="", lower=False)            # 토큰화 수행 : 문자 단위 X
eng_tokenizer.fit_on_texts(lines.eng)   # 33000개의 데이터 각 행을 토큰화
input_text = eng_tokenizer.texts_to_sequences(lines.eng)   # 단어를 숫자값 인덱스로 변환
input_text[:3]

[[28, 1], [28, 1], [28, 1]]

In [51]:
fra_tokenizer = Tokenizer(filters="", lower=False)
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:3]

[[1, 75, 8, 2], [1, 365, 3, 2], [1, 28, 512, 8, 2]]

단어장의 사이즈를 저장

In [52]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print("영어 단어장의 크기 : ", eng_vocab_size)
print("프랑스어 단어장의 크기: ", fra_vocab_size)

영어 단어장의 크기 :  4671
프랑스어 단어장의 크기:  7454


패딩추가를 위해서 최대 길이를 저장

디코더의 데이터 수정

디코더의 입력에는 <eos> 토큰이 필요없고, 디코더의 출력과 비교할 시퀀스는 <sos>가 필요하다.

In [53]:
encoder_input = input_text

# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text]
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [54]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 75, 8], [1, 365, 3], [1, 28, 512, 8]]
[[75, 8, 2], [365, 3, 2], [28, 512, 8, 2]]


In [55]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [56]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이 : ', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이 : ', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4671
프랑스어 단어장의 크기 : 7454
영어 시퀀스의 최대 길이 :  8
프랑스어 시퀀스의 최대 길이 :  17


패딩 추가

In [57]:
encoder_input = pad_sequences(encoder_input, maxlen=max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [58]:
eng_to_index = eng_tokenizer.word_index
index_to_eng = eng_tokenizer.index_word

fra_to_index = fra_tokenizer.word_index
index_to_fra = fra_tokenizer.index_word

데이터셋 나누기

데이터를 나누기 전에 먼저 한번 섞어준 후에 Training 3만개, Test 3천개로 나눈다.

In [59]:
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [60]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[18789  4004  1104 ... 15708 28521 12618]


In [61]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [62]:
encoder_input_train = encoder_input[:30000]
decoder_input_train = decoder_input[:30000]
decoder_target_train = decoder_target[:30000]

encoder_input_test = encoder_input[30000:]
decoder_input_test = decoder_input[30000:]
decoder_target_test = decoder_target[30000:]

In [63]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


모델 훈련

필요한 라이브러리 import


In [64]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [65]:
# LSTM의 출력 차원
latent_dim = 32

인코더 설계

Masking은 패딩 토큰의 숫자 0의 경우에는 연산을 제외하는 역할을 수행

In [67]:
# 인코더 설계
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, latent_dim)(encoder_inputs) # 임베딩층
enc_masking = Masking(mask_value=0.0)(enc_emb)  # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True)  # 상태값 리턴
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)  # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c]  # 인코더의 은닉 상태외 셀 상태를 저장

디코더 설계

In [68]:
# 디코더
decoder_inputs = Input(shape=(None, ))
dec_emb_layer = Embedding(fra_vocab_size, latent_dim)  # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs)  # 패딩 0은 언제나 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequence는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

# 인코더의 은닉 상태를 초기 은닉 상태 (initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점에 결과에 대해 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

학습

In [69]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     149472      input_10[0][0]                   
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     238528      input_11[0][0]                   
____________________________________________________________________________________________

In [70]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
235/235 [==============================] - 14s 33ms/step - loss: 3.8018 - acc: 0.6249 - val_loss: 2.0191 - val_acc: 0.6303
Epoch 2/50
235/235 [==============================] - 6s 24ms/step - loss: 1.7756 - acc: 0.6972 - val_loss: 1.6063 - val_acc: 0.7506
Epoch 3/50
235/235 [==============================] - 6s 25ms/step - loss: 1.5339 - acc: 0.7539 - val_loss: 1.4819 - val_acc: 0.7641
Epoch 4/50
235/235 [==============================] - 6s 25ms/step - loss: 1.4285 - acc: 0.7661 - val_loss: 1.4029 - val_acc: 0.7731
Epoch 5/50
235/235 [==============================] - 6s 24ms/step - loss: 1.3582 - acc: 0.7774 - val_loss: 1.3486 - val_acc: 0.7851
Epoch 6/50
235/235 [==============================] - 6s 25ms/step - loss: 1.3049 - acc: 0.7886 - val_loss: 1.3050 - val_acc: 0.7945
Epoch 7/50
235/235 [==============================] - 6s 25ms/step - loss: 1.2600 - acc: 0.7959 - val_loss: 1.2701 - val_acc: 0.7989
Epoch 8/50
235/235 [==============================] - 6s 24ms/step -

모델 테스트

인코더

In [71]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 32)          149472    
_________________________________________________________________
masking_1 (Masking)          (None, None, 32)          0         
_________________________________________________________________
lstm_3 (LSTM)                [(None, 32), (None, 32),  8320      
Total params: 157,792
Trainable params: 157,792
Non-trainable params: 0
_________________________________________________________________


디코더

In [72]:
# 디코더 설계
# 이전 시점의 상태롤 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# train 때 사용했던 임베딩 층을 재사용..
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단여 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [73]:
# 디코더 정의
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [74]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_fra[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

결과 확인을 위한 함수

In [75]:
def seq2eng(input_seq):
    temp = ''
    for i in input_seq :
        if(i!=0):
            temp = temp + index_to_eng[i] + ' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2fra(input_seq):
    temp = ''
    for i in input_seq:
        if ((i!=0 and i!=fra_to_index['<sos>']) and i!=fra_to_index['<eos>']):
            temp = temp + index_to_fra[i] + ' '
    return temp

훈련 데이터에 대해서 임의로 선택한 인덱스의 샘플 결과를 출력



In [76]:
for seq_index in [3, 50, 100, 600, 2005]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("원문 : ", seq2eng(encoder_input_train[seq_index]))
    print("번역문 : ", seq2fra(decoder_input_train[seq_index]))
    print("예측문 : ", decoded_sentence[:-5])
    print('\n')

원문 :  let s be objective . 
번역문 :  soyons objectives . 
예측문 :   soyez nous . 


원문 :  relax . 
번역문 :  d tends toi . 
예측문 :   d je vous prie ! 


원문 :  was tom hurt ? 
번역문 :  tom a t il t bless ? 
예측문 :   tom tait il ? 


원문 :  i feel sick . 
번역문 :  je me sens mal . 
예측문 :   je me sens en train de cie . 


원문 :  we broke up . 
번역문 :  nous nous sommes s par s . 
예측문 :   nous nous sommes deux . 




정리

validation loss가 안정적으로 떨어지면서 0.9354까지 떨어지면서 학습 중간에 Overfitting은 관찰되지 않았다.

또 테스트용 디코더 input을 2개로 받아오면서 이전에 학습할때 사용한 디코더와는 구조가 다르다. 원래 들어오는 input + input의 상태
와 이전 시점에서 나오는 outputs과 그 상태에서 다르다.
영어와 프랑스어 번역을 진행해 보았는데 이렇게 스페인어도 도전해보고 싶다. 간단한 사전, 회화 어플리케이션이 이렇게 만들어지는 것 같아 보이는데 더 나아가 장문의 텍스트를 번역하려면 데이터 양이 방대할 것이라는 생각이 든다.